# Send Custom Metrics and Test Results

In [1]:
# Quick hack to load local SDK code
import os

os.chdir(os.path.join(os.getcwd(), ".."))

# Load API key and secret from environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [2]:

import validmind as vm

vm.init(
  api_host = "http://localhost:3000/api/v1/tracking",
  api_key = "e22b89a6b9c2a27da47cb0a09febc001",
  api_secret = "a61be901b5596e3c528d94231e4a3c504ef0bb803d16815f8dfd6857fac03e57",
  project = "..."
)
  

True

## Send Custom Metrics

It is possible to send custom metrics to ValidMind without implementing a `Metric` class. The only requirement is to construct an instance of a `MetricResult`
that can be sent to the ValidMind API.

In [3]:
from validmind.vm_models import MetricResult

accuracy_metric = MetricResult(
    type="evaluation",
    scope="test_dataset",    
    key="my_custom_accuracy",
    value=0.666
)

In [4]:
vm.log_metrics([accuracy_metric])

True

## Send Custom Test Results

It is possible to send custom test results to ValidMind without implementing a `ThresholdTest` class. The only requirement is to construct an instance of a `TestResults`
that can be sent to the ValidMind API.

In [5]:
from validmind.vm_models import TestResult, TestResults

custom_params = {
    "min_percent_threshold": 0.5
}

custom_test_result = TestResults(
    category="model_performance",
    test_name="accuracy_threshold",
    params=custom_params,
    passed=False,
    results=[
        TestResult(
            passed=False,
            values={
                "score": 0.15,
                "threshold": custom_params["min_percent_threshold"],
            },
        )
    ],
)

In [6]:
vm.log_test_results([custom_test_result])

True

## Implementing Custom Metrics or ThresholdTests

It is possible to implement custom metrics or threshold test classes. The are only two requirements for getting this to work:

- We need to build a `TestPlan` that can execute the custom metric or threshold test (or add it to an existing `TestPlan`, TBD).
- We need to implement a `run` method on the custom metric or threshold test class.

### Implementing a Custom Metric

The following example shows how to implement a custom metric that calculates the mean of a list of numbers.

In [7]:
from dataclasses import dataclass
from validmind.vm_models import Metric

@dataclass
class MeanMetric(Metric):
    type = "dataset"
    key = "mean_of_values"

    def run(self):
        if "values" not in self.params:
            raise ValueError("values must be provided in params")

        if not isinstance(self.params["values"], list):
            raise ValueError("values must be a list")
        
        values = self.params["values"]
        mean = sum(values) / len(values)
        return self.cache_results(mean)

### Implementing a Custom ThresholdTest

The following example shows how to implement a custom threshold test that fails if the mean of a list of numbers is greater than 5.

In [8]:
from dataclasses import dataclass
from validmind.vm_models import ThresholdTest

@dataclass
class MeanThresholdTest(ThresholdTest):
    category = "data_quality"
    name = "mean_threshold"
    default_params = {"mean_threshold": 5}    

    def run(self):
        if "values" not in self.params:
            raise ValueError("values must be provided in params")

        if not isinstance(self.params["values"], list):
            raise ValueError("values must be a list")
        

        values = self.params["values"]
        mean = sum(values) / len(values)        

        passed = mean <= self.params["mean_threshold"]
        results = [
            TestResult(
                passed=passed,
                values={
                    "mean": mean,
                    "values": values,
                },
            )
        ]

        return self.cache_results(results, passed=passed)

### Implementing a Custom TestPlan

The following example shows how to implement a custom test plan that executes the custom metric and threshold test.

In [9]:
from validmind.vm_models import TestPlan

class MyCustomTestPlan(TestPlan):
    """
    Custom test plan
    """

    name = "my_custom_test_plan"
    required_context = []
    tests = [MeanMetric, MeanThresholdTest]

my_custom_test_plan = MyCustomTestPlan(config={
    "mean_of_values": {
        "values": [1, 2, 3, 4, 5]
    },
    "mean_threshold": {
        "values": [6, 7, 8, 9, 10]
    }
})
my_custom_test_plan.run()